In [31]:
from geopy.geocoders import Nominatim
import requests
import pandas as pd
from tqdm.notebook import tqdm
import time

In [2]:
building_type_translation = {
    "residential": "Жилое здание",
    "commercial": "Коммерческое здание",
    "administrative": "Административное здание",
    "house": "Частный дом",
    "industrial": "Промышленное здание",
    "retail": "Торговое здание",
    "yes": "Здание",
    "apartments": "Многоквартирный дом",
    "unknown": "Неизвестный тип здания",
    "church": "Церковь",
    "university": "Университет",
    "garages": "Гаражи",
    "bakehouse": "Пекарня",
    "terrace": "Терраса",
    "service": "Сервисное здание",
    "detached": "Отдельно стоящее здание",
    "office": "Офис",
    "shed": "Сарай",
    "dormitory": "Общежитие",
    "warehouse": "Склад",
    "school": "Школа",
    "sports_centre": "Спортивный центр",
    "gatehouse": "Караульное помещение",
    "construction": "Строящийся объект",
    "hospital": "Больница",
    "hotel": "Гостиница",
    "garage": "Гараж",
    "pavilion": "Павильон",
    "kindergarten": "Детский сад",
    "roof": "Кровля",
    "manufacture": "Производственное здание",
    "train_station": "Железнодорожный вокзал",
    "barrack": "Казарма",
    "shop": "Магазин",
    "college": "Колледж",
    "error": "Неизвестный тип здания",
    "stands": "Трибуны",
    "hangar": "Ангар",
    "supermarket": "Супермаркет",
    "government": "Государственное здание",
    "factory": "Фабрика",
    "hut": "Хижина",
    "clinic": "Клиника",
    "public": "Общественное здание",
    "parking": "Парковка",
    "store": "Склад",
    "brewery": "Пивоварня",
    "abandoned": "Заброшенное здание",
    "civic": "Гражданское здание"
}

In [28]:
def get_building_type(address):
    try:
        # Геокодируем адрес с помощью Nominatim
        geolocator = Nominatim(user_agent="building_type_finder")
        location = geolocator.geocode(address)
    
        if not location:
            return "unknown"
    
        # Получаем координаты из геокодирования
        lat, lon = location.latitude, location.longitude
    
        # Формируем запрос к Overpass API для поиска типа здания
        overpass_url = "http://overpass-api.de/api/interpreter"
        overpass_query = f"""
        [out:json];
        way(around:50,{lat},{lon})["building"];
        out tags;
        """
        response = requests.get(overpass_url, params={'data': overpass_query})
        data = response.json()
    
        # Проверяем, есть ли здания в результате
        if not data.get('elements'):
            return "unknown"
    
        for element in data['elements']:
            building_type = element['tags'].get("building", "Иное")
            return building_type
        
        # Если цикл не возвращает значение
        return "unknown"
    
    except Exception as e:
        print(f"Ошибка: {e}")
        return "unknown"

In [30]:
# Пример использования
address = "Москва Маршала Василевского 17"
building_type = get_building_type(address)
print(f"Тип здания: {building_type}")

Тип здания: apartments


In [35]:
df_input = pd.read_csv("normalization_in (8).csv", dtype="str", sep=";").astype("str")
df_input["id"] = df_input["id"].astype(int)

In [69]:
df_input["тип_здания"] = ""
df_input["building_type"] = ""

In [ ]:
for index, row in tqdm(df_input.iterrows()):
    i = index
    address = str(row["address"])
    building_type, building_type_original = get_building_type(address)
    df_input.at[index, "тип_здания"] = building_type
    df_input.at[index, "тип_здания"] = building_type_original
    # time.sleep(2)

In [79]:
df_input["building_type"] = df_input["тип_здания"]

In [80]:
df_input

,id,address,тип_здания,building_type
0,1,"ЧЕБАРКУЛЬ, 9 МАЯ, 27",apartments,apartments
1,2,"БАЙМАКСКИЙ, ИШЕЙ, ШКОЛЬНАЯ, Д. 1",yes,yes
2,3,"ОЗЕРЫ, ПОЛУРЯДИНКИ, Д. 6",unknown,unknown
3,4,"СОСНОВЫЙ БОР, СОЛНЕЧНАЯ, 30",apartments,apartments
4,5,"МАГНИТОГОРСК, КАРЛА МАРКСА, 13",apartments,apartments
...,...,...,...,...
9995,9996,"КАРАБУДАХКЕНТСКИЙ, КАРАБУДАХКЕНТ, ЖУКОВСКОГО, 5",house,house
9996,9997,"МОСКВА, ЛЮБЛИНСКАЯ, Д. 17,К. 2",unknown,unknown
9997,9998,"ЯРОСЛАВЛЬ, ЛЕСКОВА, Д. 23",construction,construction
9998,9999,"ПОЧИНКОВСКИЙ, ПОЧИНКИ, СТРОИТЕЛЕЙ, Д. 3",house,house


In [92]:
df_building_types = pd.DataFrame(
    list(building_type_translation.items()),
    columns=["building_type", "тип_здания_ru"]
)

In [93]:
res = pd.merge(df_input, df_building_types, how="left", on="building_type")

In [94]:
res["building_type"][res["тип_здания_ru"].isna()].unique()

array([], dtype=object)

In [102]:
res.to_excel("res.xlsx")